# 第11章 Webスクレイピング
## 11.1 プロジェクト:webbrowserモジュールを用いたマップ検索

In [1]:
import webbrowser, sys, pyperclip

address = input()
webbrowser.open('https://www.google.com/maps/place/' + address) #urlをオープン

Hiroshima


True

## 11.2 requestsモジュールを用いたWebサイトからのファイルのダウンロード
### 11.2.1 requests.get()関数を用いたWebページのダウンロード

In [2]:
import requests

res=requests.get('http://automatetheboringstuff.com/files/rj.txt')
type(res)

requests.models.Response

In [3]:
res.status_code==requests.codes.ok #ダウンロード成功

True

In [4]:
len(res.text) #文字数

178978

In [5]:
print(res.text[0:250]) #250文字出力

The Project Gutenberg EBook of Romeo and Juliet, by William Shakespeare

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Projec


### 11.2.2 エラーをチェックする

In [6]:
res=requests.get('http://automatetheboringstuff.com/page_that_does_not_exist')
res.raise_for_status() #ファイルのダウンロードが失敗に終われば例外を起こす

HTTPError: 404 Client Error: Not Found for url: http://automatetheboringstuff.com/page_that_does_not_exist

In [7]:
#ダウンロードの失敗がプログラムを停止させるほどでなければ、例外処理をする
res=requests.get('http://automatetheboringstuff.com/page_that_does_not_exist')
try:
    res.raise_for_status() #ファイルのダウンロードが失敗に終われば例外を起こす
except Exception as exc:
    print('問題あり:{}'.format(exc))
    
print('プログラムの続行が可能')

問題あり:404 Client Error: Not Found for url: http://automatetheboringstuff.com/page_that_does_not_exist
プログラムの続行が可能


### 11.2.3 ダウンロードしたファイルをハードドライブに保存する

In [8]:
folder='C:\\Users\\nakam\\Python\\Data\\Boring_Python\\11.2.3\\'

import requests
res=requests.get('http://automatetheboringstuff.com/files/rj.txt')
res.raise_for_status() #ダウンロード成功を確認
play_file=open(folder+'RomeoAndJuliet.txt','wb') #バイナリ書き込みモードで開く

for chunk in res.iter_content(100000): #100000バイトずつ書き込むようループ
    play_file.write(chunk)

play_file.close()

## 11.3 HTML

In [9]:
# <>で記述されたものをタグ
# 開始タグと終了タグでテキストを囲むと要素になる
# <strong>Hello</strong> world! : 開始タグは囲んだテキストを強調表示することを意味する 
                                 #終了タグは協調の終了を表す 

In [10]:
# <a href="http://xxxx-">python books</a>  : aタグはリンクとなるテキストを囲むもの
                                            #リンク先のurlはhref属性に記述する 
    
# <div><span>タグはそれ自身は特に意味を持っていないが、囲んだ範囲をひと固まりとする

#id属性、class属性はどちらも名前を割り当てることが役割
#id属性 : 1ページに1回しか同じ名前は使えない
#class属性 : 何回でも同じ名前を使える

In [11]:
#div class="example"> : このグループのクラスをexampleに設定(exampleは以下で定義)
#  <p>例文です</p> : pタグは段落を表す
#  <p>例文です</p>
#</div>

#.example {
#  width: 40px;/*幅を40pxに*/
#  height: 200px;/*高さを200pxに*/
#  background:orange;/*背景色をオレンジに*/
#  }

## 11.4 BeautifulSoupモジュールを用いたHTMLの解析

In [12]:
!pip install beautifulsoup4

### 11.4.1 HTMLからBeautifulSoupオブジェクトを生成する

In [13]:
import requests, bs4
res=requests.get('http://nostarch.com')
res.raise_for_status()
no_starch_soup=bs4.BeautifulSoup(res.text)
type(no_starch_soup)

bs4.BeautifulSoup

In [14]:
#HTMLファイルを読み込む場合は以下
folder='C:\\Users\\nakam\\Python\\Data\\Boring_Python\\11.4.1\\'

example_file=open(folder+'example.html')
example_soup=bs4.BeautifulSoup(example_file)
type(example_soup)

bs4.BeautifulSoup

### 11.4.2 select()メソッドを用いて要素を見つける
CSS:ウェブページのスタイルを指定する言語

In [15]:
#以下の要素にマッチさせる方法
#soup.select('div') : すべての<div>要素
#soup.select('#author') : id属性がauthorである要素
#soup.select('.notice') : CSSクラス属性がnoticeである全要素
#soup.select('div span') : <div要素>の中のすべての<span>要素
#soup.select('div > span') : <div要素>直下のすべての<span>要素
#soup.select('input[name]') : name属性を持つすべての<input>要素
#soup.select('input[type="button"]') : type属性の値がbuttonであるすべての<input>要素

In [16]:
elems=example_soup.select('#author') #id属性がauthorである要素にマッチ
type(elems)

list

In [17]:
len(elems) #tagオブジェクトの数

1

In [18]:
type(elems[0])

bs4.element.Tag

In [19]:
elems[0].getText() #テキストを抽出

'Al Sweigart'

In [20]:
str(elems[0]) #開始タグ、終了タグを含む文字列

'<span id="author">Al Sweigart</span>'

In [21]:
elems[0].attrs

{'id': 'author'}

In [22]:
#p要素を取り出す場合は以下

p_elems=example_soup.select('p') #id属性がauthorである要素にマッチ
print(len(p_elems))
print(p_elems[0])

3
<p>Download my <strong>Python</strong> book from <a href="http://inventwithpython.com">my website</a>.</p>


### 11.4.3 要素の属性からデータを取得する
get()メソッド

In [23]:
span_elem=example_soup.select('span')[0] #span要素の1つ目を取得
span_elem

<span id="author">Al Sweigart</span>

In [24]:
span_elem.get('id') #属性名idの属性を検索

'author'

In [25]:
span_elem.attrs

{'id': 'author'}

## 11.5 プロジェクト：Google検索

In [26]:
import requests, sys, webbrowser
from bs4 import BeautifulSoup as bs

print('検索キーワードを入力してください')
keyword=input()

print('Googling...')  # Googleページをダウンロード中にテキストを表示
res = requests.get('http://google.com/search?q='+keyword)
res.raise_for_status()

res = res.text
soup = bs(res,"html.parser")
list=[]

tags = soup.find_all("div", class_="ZINbbc xpd O9g5cc uUPGi")

num_open = 3 #検索数
number=0

for i in tags:
    if number==3:
        break
    
    number=number+1
    
    if(i.find("div", class_="BNeawe s3v9rd AP7Wnd") != None):
        print(i.find("a").get("href")) #urlの出力
        list.append(i.find("a").get("href"))
        print('')             
        print(i.find("div", class_="BNeawe s3v9rd AP7Wnd").string) #サイトの説明
        
    print('')
    print('')

# 各結果をブラウザのタブで開く
#for i in range(num_open):
#    print(list[i])
#    webbrowser.open('http://google.com' + list[i])

検索キーワードを入力してください
python
Googling...
/imgres?imgurl=https://upload.wikimedia.org/wikipedia/commons/f/f8/Python_logo_and_wordmark.svg&imgrefurl=https://ja.wikipedia.org/wiki/%25E3%2583%2595%25E3%2582%25A1%25E3%2582%25A4%25E3%2583%25AB:Python_logo_and_wordmark.svg&h=153&w=518&tbnid=YScuu7LWBQP5SM:&q=python&tbnh=59&tbnw=200&usg=AI4_-kTZicjCQJEIHp46sxvIeoVf4a6w9Q&vet=1&docid=XqaRwRVuoWR6bM&itg=1&sa=X&ved=2ahUKEwi30sWkz4DnAhWpITQIHYNlBYMQ9QF6BAgLEAQ

None


/url?q=https://www.python.jp/&sa=U&ved=2ahUKEwi30sWkz4DnAhWpITQIHYNlBYMQFjAVegQICBAB&usg=AOvVaw3MlYdgSosVsV5MZNzAj4Av

None


/url?q=https://ja.wikipedia.org/wiki/Python&sa=U&ved=2ahUKEwi30sWkz4DnAhWpITQIHYNlBYMQFjAWegQIChAB&usg=AOvVaw37SqzD28BED6mBtZloN-Hx

None




## 11.6 画像を保存し前のコミックを見つける

In [27]:
import requests, os, bs4
import time

folder='C:\\Users\\nakam\\Python\\Data\\Boring_Python\\11.6'
os.chdir(folder)

url = 'http://xkcd.com/'               # 開始URL
os.makedirs('xkcd', exist_ok=True)    # ./xkcdに保存する

max_number=5 #ダウンロード回数を定義
number=0

while not url.endswith('#'):
    number=number+1
    
    if number==max_number:
        break

    # ページをダウンロードする
    print('ページをダウンロード中 {}...'.format(url))
    res = requests.get(url)
    res.raise_for_status()

    soup = bs4.BeautifulSoup(res.text)

    # コミック画像のURLを見つける
    #<div id="comic">
    #<img src="//imgs.xkcd.com/comics/star_wars_voyager_1.png" title=
    #"There's some flexibility depending on your standards for measuring 
    #runtime and the various special editions. If you still want to have 
    #a party, I'm sure you can find some combination that works." alt=
    #"Star Wars Voyager 1" srcset="//imgs.xkcd.com/comics/star_wars_
    #voyager_1_2x.png 2x">
    #</div>

    
    comic_elem = soup.select('#comic img')
    if comic_elem == []:
        print('コミック画像が見つかりませんでした。')
    else:
        comic_url = 'http:' + comic_elem[0].get('src')
        # 画像をダウンロードする
        print('画像をダウンロード中 {}...'.format(comic_url))
        res = requests.get(comic_url)
        res.raise_for_status()

        # 画像を./xkcdに保存する
        image_file = open(os.path.join('xkcd', os.path.basename(comic_url)), 'wb')
        for chunk in res.iter_content(100000):
            image_file.write(chunk)
        image_file.close()
        
    # PrevボタンのURLを取得する
    #<a rel="prev" href="/2252/" accesskey="p">&lt; Prev</a>
    
    prev_link = soup.select('a[rel="prev"]')[0]
    url = 'http://xkcd.com' + prev_link.get('href')

print('完了')

ページをダウンロード中 http://xkcd.com/...
画像をダウンロード中 http://imgs.xkcd.com/comics/star_wars_voyager_1.png...
ページをダウンロード中 http://xkcd.com/2252/...
画像をダウンロード中 http://imgs.xkcd.com/comics/parenthetical_names.png...
ページをダウンロード中 http://xkcd.com/2251/...
画像をダウンロード中 http://imgs.xkcd.com/comics/alignment_chart_alignment_chart.png...
ページをダウンロード中 http://xkcd.com/2250/...
画像をダウンロード中 http://imgs.xkcd.com/comics/ok_okay_ok.png...
完了


## 11.7 seleniumモジュールを用いたブラウザの制御

In [28]:
!pip install chromedriver-binary==79.0.3945.36.0

In [29]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import chromedriver_binary

browser=webdriver.Chrome()
browser.get('http://inventwithpython.com')

### 11.7.2 ページの要素を見つける

In [30]:
#複数とマッチさせたい場合はfind_elementsとする
#browser.find_element_by_class_name(name) : CSSクラスがnameである要素
#browser.find_element_by_css_selector(selector) : CSSセレクタにマッチする要素
#browser.find_element_by_link_text(text) : textに完全一致する<a>要素
#browser.find_element_by_partial_link_text(text) : textに部分一致する<a>要素
#browser.find_element_by_name(name) : name属性が一致する要素
#browser.find_element_by_tag_name(name) : タグ名が名前に一致する要素（大文字小文字区別無し）

In [31]:
#上記でWebElementオブジェクトを取得したら以下のメソッドを呼び出し可能
#tag_name : タグ名。例えば<a>要素なら'a'
#get_attribute(name) : 要素の属性nameの値
#text : 要素の内部テキスト
#clear() : inputやtextarea要素のテキストを消去する
#is_displayed() : 要素が歌詞ならTrue,そうでなければFalseを返す
#is_enabled() : input要素が有効ならTrue、無効ならFalseを返す
#is_selected() : checkboxやradiobutton要素がチェックされていればTrue、されていなければFalseを返す
#location : 'X'と'Y'のキーを持つ辞書で、要素のページ上の座標を返す

In [33]:
browser=webdriver.Chrome()
browser.get('http://inventwithpython.com')
try:
    elem=browser.find_element_by_class_name('card-img-top')
    print('そのクラス名を持つ要素 <{}>を見つけた'.format(elem.tag_name))
except:
    print('そのクラス名を持つ要素は見つからなかった')

そのクラス名を持つ要素 <img>を見つけた


### 11.7.3 ページをクリックする
click()メソッド

In [34]:
#WebElementオブジェクトにclick()メソッドがある
link_elem=browser.find_element_by_link_text('Read Online for Free')
link_elem.click()

### 11.7.4 フォームを記入して送信する
send_keys()メソッド

In [35]:
#elem.send_keys('12345') #入力
#elem.submit() #送信ボタンを記入

### 11.7.5 特殊なキーを送信する

In [36]:
#Keys.Down / Keys.Up / Keys.LEFT / Keys.RIGHT : 矢印
#Keys.ENTER / Keys.RETURN
#Keys.HOME / Keys.END / Keys.PAGE_DOWN / Keys.PAGE_UP
#Keys.ESCAPE / Keys.BACK_SPACE / Keys.DELETE
#Keys.F1
#Keys.TAB

In [37]:
#ページをスクロールする
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

browser=webdriver.Chrome()
browser.get('http://nostarch.com')
html_elem=browser.find_element_by_tag_name('html') #キー入力する際にはhtml中身全体を取得する
html_elem.send_keys(Keys.END)
html_elem.send_keys(Keys.HOME)

## 11.7.6 ブラウザのボタンをクリックする

In [38]:
#browser.back() : 戻るボタンをクリック
#browser.forward() : 進むボタンをクリック
#browser.refresh() : 再読み込みボタンをクリック
#browser.quit() : ウインドウを閉じるボタンをクリックする

## 11.10 演習プロジェクト
### 11.10.1 電子メーラー

In [43]:
#Gmailに自動ログインし、メール送信

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import time
import sys

to_address = 'nakamukaiya@yahoo.co.jp'
message = 'test'

MY_ADDRESS = 'nakamukaiya@gmail.com' 
MY_PASSWORD = 'MHIh@ru0701' 

browser=webdriver.Chrome()
browser.get('https://www.google.com/gmail/')

# IDの入力
id_input = browser.find_element_by_id('identifierId')
#<input type="email" class="whsOnd zHQkBf" jsname="YPqjbf" autocomplete="username" 
#spellcheck="false" tabindex="0" aria-label="Email or phone" name="identifier" 
#value="" autocapitalize="none" id="identifierId" dir="ltr" data-initial-dir="ltr" 
#data-initial-value="">

id_input.send_keys(MY_ADDRESS)
next = browser.find_element_by_id('identifierNext')
#<div role="button" id="identifierNext" class="U26fgb O0WRkf zZhnYe e3Duub C0oVfc FliLIb DL0QTb M9Bg4d" 
#jscontroller="VXdfxd" jsaction="click:cOuCgd; mousedown:UX7yZ; mouseup:lbsD7e; mouseenter:tfO1Yc; 
#mouseleave:JywGue; focus:AHmuwe; blur:O22p3e; contextmenu:mg9Pef;touchstart:p6p2H; touchmove:FwuNnf; 
#touchend:yfqBxc(preventMouseEvents=true|preventDefault=true); touchcancel:JMtRjd;" jsshadow="" 
#jsname="Njthtb" aria-disabled="false" tabindex="0"><div class="Vwe4Vb MbhUzd" 
#jsname="ksKsZd"></div><div class="ZFr60d CeoRYc"></div><span jsslot="" class="CwaK9">
#<span class="RveJvd snByac">Next</span></span></div>


next.click()
# ちょっと待つ。
time.sleep(5)

# パスワードの入力
password_input = browser.find_element_by_name('password')
#<input type="password" class="whsOnd zHQkBf" jsname="YPqjbf" autocomplete="current-password" 
#spellcheck="false" tabindex="0" aria-label="Enter your password" name="password" 
#autocapitalize="off" dir="ltr" data-initial-dir="ltr" data-initial-value="">

password_input.send_keys(MY_PASSWORD)
next = browser.find_element_by_id('passwordNext')
next.click()
time.sleep(15)

In [44]:
# 「作成」ボタンを押す
divs = browser.find_elements_by_xpath("//div[@role='button']")
#<div class="T-I J-J5-Ji T-I-KE L3" role="button" tabindex="0" jslog="20510; 
#u014N:cOuCgd,Kr2w4b" gh="cm" style="user-select: none;">作成</div>

for d in divs:
    if d.text == '作成':
        d.click()
        break
time.sleep(5)

# To: を入力
to_textarea = browser.find_element_by_name('to')
#<textarea rows="1" id=":r2" class="vO" name="to" spellcheck="false" 
#autocomplete="false" autocapitalize="off" autocorrect="off" tabindex="1" 
#dir="ltr" aria-label="To" role="combobox" aria-autocomplete="list" 
#style="width: 311px;"></textarea>

to_textarea.send_keys(to_address + '\n')
time.sleep(1)

# 件名を入力
subject_input = browser.find_element_by_name('subjectbox')
#<input name="subjectbox" id=":qk" class="aoT" 
#autocomplete="off" spellcheck="true" tabindex="1" 
#placeholder="件名" aria-label="件名">

subject_input.send_keys('自動送信')
time.sleep(1)

# メッセージ本文を入力
message_div = browser.find_element_by_xpath("//div[@aria-label='メッセージ本文']")
#<div id=":rp" class="Am Al editable LW-avf tS-tW" 
#hidefocus="true" aria-label="メッセージ本文" g_editable="true" 
#role="textbox" aria-multiline="true" contenteditable="true" 
#tabindex="1" style="direction: ltr; min-height: 269px;" itacorner="6,7:1,1,0,0"><br></div>

message_div.send_keys(message)
time.sleep(5)

# 「送信」ボタンを押す
divs = browser.find_elements_by_xpath("//div[@role='button']")
#<div id=":qa" class="T-I J-J5-Ji aoO v7 T-I-atl L3" 
#role="button" tabindex="1" data-tooltip="送信 ‪(Ctrl+Enter)‬" 
#aria-label="送信 ‪(Ctrl+Enter)‬" data-tooltip-delay="800" jslog="32601; 
#u014N:cOuCgd,Kr2w4b; dYFj7e:true; 
#11:WyIjbXNnLWE6ci00MjE3MzAwMTYxNDg0MDc4NTQ0IixudWxsLG51bGwsbnVsbCwxLG51b
#GwsWyIjdGhyZWFkLWE6ci00MjEyMzQyNzA3NTM5MDU4ODY0Il0sZmFsc2Vd" 
#style="user-select: none;">送信</div>

for d in divs:
    if d.text == '送信':
        d.click()
        break

## 11.10.2 画像サイトのダウンローダー

In [45]:
#Imgurページの例
#①キーワードを入力
#②フォルダ名を入力
#指定したページの画像を指定したフォルダへ保存する

import requests, os, sys, bs4

# urlを以下に入力
word=input()
word=word.replace(' ','+') #空白を+に置換
url='https://imgur.com/search/score?q='+word

#フォルダ名を指定
os.chdir('C:\\Users\\nakam\\Python\\Data\\Boring_Python\\11.10.2')
print('フォルダ名を入力してください')
folder=input()

#フォルダ作成
os.makedirs(folder,exist_ok=True)

#url情報を取得
res=requests.get(url)
res.raise_for_status()
    
soup=bs4.BeautifulSoup(res.text)

#imgリンクを取得(src=img)
link_elems=soup.select('img')

for i in range(0,len(link_elems)):  
    try:
        #ファイル保存
        print('{}を保存しています...'.format(link_elems[i].get('src')))
        image_file=open(os.path.join(folder,os.path.basename('https:'+link_elems[i].get('src'))),'wb') #https:を追加する必要がある
            
        #requests.getでWebページをダウンロード
        res=(requests.get('https:'+link_elems[i].get('src'))) #https:を追加する必要がある
                
        for chunk in res.iter_content(100000):
            image_file.write(chunk)
        
        image_file.close()
        
    except OSError: 
        print('{}は保存できないのでスキップします...'.format(link_elems[i].get('src')))

print('ダウンロード完了しました')
res.close()

Game of Thrones
フォルダ名を入力してください
GOT
//i.imgur.com/JLWVcKxb.jpgを保存しています...
//i.imgur.com/Q7v2qWvb.jpgを保存しています...
//i.imgur.com/eFtGFtrb.jpgを保存しています...
//i.imgur.com/CZEczqib.jpgを保存しています...
//i.imgur.com/QNNH4tjb.jpgを保存しています...
//i.imgur.com/u43MKj7b.jpgを保存しています...
//i.imgur.com/HB11hmXb.jpgを保存しています...
//i.imgur.com/fuQohQLb.jpgを保存しています...
//i.imgur.com/0cqiSePb.jpgを保存しています...
//i.imgur.com/lZYJKVcb.jpgを保存しています...
//i.imgur.com/6xwTT2Pb.jpgを保存しています...
//i.imgur.com/GpkSbuSb.jpgを保存しています...
//i.imgur.com/1DrIcsSb.jpgを保存しています...
//i.imgur.com/06h6TCSb.jpgを保存しています...
//i.imgur.com/hBTIX8kb.jpgを保存しています...
//i.imgur.com/5pci7Knb.jpgを保存しています...
//i.imgur.com/8wN5yB5b.jpgを保存しています...
//i.imgur.com/ft5t54ab.jpgを保存しています...
//i.imgur.com/5FVblmhb.jpgを保存しています...
//i.imgur.com/AHgGOprb.jpgを保存しています...
//i.imgur.com/THrIHsUb.jpgを保存しています...
//i.imgur.com/61MBzc6b.jpgを保存しています...
//i.imgur.com/vNYm9Qqb.jpgを保存しています...
//i.imgur.com/gSIVWtZb.jpgを保存しています...
//i.imgur.com/MMw3BCAb.jpgを保存しています...
//i.imgur.com/3

## 11.10.3 2048

In [46]:
#2048ゲームの自動操作

import requests, os, sys, bs4
from selenium import webdriver
import chromedriver_binary
from selenium.webdriver.common.keys import Keys
from time import sleep

# urlを指定
url='https://play2048.co/'

#Goole Chrome起動
browser=webdriver.Chrome()
browser.get(url)

#操作(下左上右を繰り返し)
html_elem=browser.find_element_by_tag_name('html') #html全体を選択

for n in range(0,20):
    html_elem.send_keys(Keys.DOWN)
    sleep(0.2)
    html_elem.send_keys(Keys.LEFT)
    sleep(0.2)
    html_elem.send_keys(Keys.UP)
    sleep(0.2)
    html_elem.send_keys(Keys.RIGHT)
    sleep(0.2)

## 11.10.4 リンクの検査

In [47]:
import requests, os, sys, bs4, re

# urlを指定
url='http://www.kuremitsuta-h.hiroshima-c.ed.jp/web_mitsuta/index2.html'
url2='http://www.kuremitsuta-h.hiroshima-c.ed.jp/web_mitsuta'

#フォルダ名を指定
os.chdir('C:\\Users\\nakam\\Python\\Data\\Boring_Python\\11.10.4')
folder='Link_Inspection'

#フォルダ作成
os.makedirs(folder,exist_ok=True)

#url情報を取得
res=requests.get(url)
res.raise_for_status()
    
soup=bs4.BeautifulSoup(res.text,'lxml')

#aタグを検索する
link_elems=soup.select('a')

href_set=set() #セットを定義（Listと異なり重複しない）

for link in link_elems:
    #href属性を取得する
    href=link.get('href')

    if not href: #空の場合は抜ける
        continue
    
    #リンクがhttpで始まっていない相対パスの時、正規化する
    if not href.startswith('http'):
        href=url2+'/'+href
    
    ##ラベルを削除
    href=re.sub(r'(#.+)','',href) #置換
    
    #過去にダウンロードしたURLは省く
    if href in href_set:
        continue
    
    href_set.add(href) #Setの場合はaddで追加する
   
    print(href,end='',flush=True) #flushで結果を即時出力

    try:
        #リンク先をダウンロード
        res=requests.get(href)
        if res.status_code==404:
            print('->404 Not Found')
        elif res.status_code==200:
            #hrefの冒頭を'http(s)://をとる'
            filename=re.sub(r'^(https?://)','',href)
            #パスに使えない文字を_に変換してファイル名にする
            filename=re.sub(r'[/~?&+]','_',filename)
            hfile=open(os.path.join(folder,filename),'wb')
            for chunk in res.iter_content(100000):
                hfile.write(chunk)
            hfile.close()
            print('->',filename,'に保存')
    
    except:
        print('-> Error')

http://www.kuremitsuta-h.hiroshima-c.ed.jp/web_mitsuta/index2.html-> www.kuremitsuta-h.hiroshima-c.ed.jp_web_mitsuta_index2.html に保存
http://www.kuremitsuta-h.hiroshima-c.ed.jp/web_mitsuta/g-kotyo.html-> www.kuremitsuta-h.hiroshima-c.ed.jp_web_mitsuta_g-kotyo.html に保存
http://www.kuremitsuta-h.hiroshima-c.ed.jp/web_mitsuta/g-kouka.html-> www.kuremitsuta-h.hiroshima-c.ed.jp_web_mitsuta_g-kouka.html に保存
http://www.kuremitsuta-h.hiroshima-c.ed.jp/web_mitsuta/g-rekishi.html-> www.kuremitsuta-h.hiroshima-c.ed.jp_web_mitsuta_g-rekishi.html に保存
http://www.kuremitsuta-h.hiroshima-c.ed.jp/web_mitsuta/g-keieikeikaku.html-> www.kuremitsuta-h.hiroshima-c.ed.jp_web_mitsuta_g-keieikeikaku.html に保存
http://www.kuremitsuta-h.hiroshima-c.ed.jp/web_mitsuta/g-kokken.html-> www.kuremitsuta-h.hiroshima-c.ed.jp_web_mitsuta_g-kokken.html に保存
http://www.kuremitsuta-h.hiroshima-c.ed.jp/web_mitsuta/g-access.html-> www.kuremitsuta-h.hiroshima-c.ed.jp_web_mitsuta_g-access.html に保存
http://www.kuremitsuta-h.hiroshima-